# Phase II: Data Curation, Exploratory Analysis and Plotting (5\%)

### Team Members:
- Melanie Yu
- Yunhan Luo
- Ani McCaskill
- Owen Sweetman


## Part 1:
(1%) Expresses the central motivation of the project and explains the (at least) two key questions to be explored. Gives a summary of the data processing pipeline so a technical expert can easily follow along.

# Problem Motivation